# Installing GATK + notes
    This page will be a reference for installation, different tutorials and how to call polyploids

  #### Get GATK jar and .yml files here:
  https://github.com/broadinstitute/gatk/releases
  
  wget .zip file to a src directory, unzip, run conda install below in that direcoty
  
  #### directions for conda install here:
  https://gatk.broadinstitute.org/hc/en-us/articles/360035889851--How-to-Install-and-use-Conda-for-GATK4
  
  #### vivaswat notes
  https://microcollaborative.atlassian.net/wiki/spaces/BL/pages/73957377/Bioinformatics+for+polyploids
  
  
  
  #### IDEA: 
  
  - call snps with GATK haplotype caller. 
  - run one per sample with correct ploidy into gvcf
  - then use GenotypeGVCF to call snps across all gvcfs, no need to tell ploidy.  
  

In [49]:
import sys

#sys.path.append('/home/faske/g/anaconda3/envs/py34/lib/python3.4/site-packages')
sys.path.append('/data/gpfs/assoc/parchmanlab/tfaske/anaconda3/envs/py36/lib/python3.6/site-packages')
sys.path.append("/data/gpfs/assoc/parchmanlab/tfaske/ipynb/include_utils")

import ipyparallel as ipp
import os, time
from os import environ
import include_utils as u
import pandas as pd
import numpy as np
import scipy as sp
import numbers
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.cm as cm
import matplotlib.colors as mcolors
#import vcf
from sklearn import preprocessing
from subprocess import Popen, PIPE, call, check_output
import seaborn as sns
from IPython.display import FileLink
import urllib.request as urllib2
import dill
import traceback
from pandas import Series, DataFrame
import gzip
import warnings
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)
%config InlineBackend.figure_format = 'retina'
from Bio import SeqIO
#import pysam
from collections import OrderedDict, namedtuple, Counter
import operator
import multiprocessing as mp
import shutil
import tempfile
#from ipyparallel import Client
import scandir
import glob
from Bio.SeqIO.QualityIO import FastqGeneralIterator
import pickle
import re
from itertools import chain
#import Levenshtein as lv

In [2]:
root = "/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo"

In [3]:
snp_dir = os.path.join(root,'SNPcall')

In [4]:
bam_dir = os.path.join(snp_dir,'good_bams')

In [5]:
cd $bam_dir

/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall/good_bams


In [6]:
bam_files = []
files = !find . -type f -name '*sorted.bam'
files = [os.path.abspath(x) for x in files if 'bam' in x]
for x in files:
    bam_files.append(x)
bam_files = sorted(bam_files)

In [7]:
len(bam_files), bam_files[0]

(695,
 '/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall/good_bams/AT2_BB_10_sorted.bam')

In [26]:
assembly = "/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/assembly/reference.fasta"

#assembly indexed with dDocent

In [75]:
!samtools index {assembly}

samtools index: "/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/assembly/reference.fasta" is in a format that cannot be usefully indexed


## SNP CALL with freebayes TEST

call from dDocent: freebayes -b split.1.bam -t mapped.1.bed -v raw.1.vcf -f reference.fasta -m 5 -q 5 -E 3 --min-repeat-entropy 1 -V --populations popmap -n 10 2> fb.1.error.log

**settings to consider**:
  - -L --bam-list FILE A file containing a list of BAM files to be analyzed.
  - -A --cnv-map FILE
                   Read a copy number map from the BED file FILE, which has
                   the format:
                      reference sequence, start, end, sample name, copy number
        ex: 
        chrX -1 -1 NA12878 2  # means NA12878 has copy number 2 in all of chrX
        chrX -1 -1 NA12879 1  # ... copy number 1
        chrX -1 -1 NA13469 2
        chrX -1 -1 NA21328 2
        chrX -1 -1 NA12970 2
        chrY -1 -1 NA12877 1
        chrX -1 -1 NA12877 1
        chr20 20000 30000 NA12877 1   # NA12877 has one copy of chr20:20000..30000
        chr20 20000 30000 NA12879 3   # NA12879 has four copies of chr20:20000..30000

Do a test: move ~15 files to a test_dir


In [9]:
test_dir = os.path.join(snp_dir,'test')
testbwa_dir = os.path.join(test_dir,'good_bams')

In [10]:
cd $testbwa_dir

/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall/test/good_bams


In [11]:
bam_files = []
files = !find . -type f -name '*sorted.bam'
files = [os.path.abspath(x) for x in files if 'bam' in x]
for x in files:
    bam_files.append(x)
bam_files = sorted(bam_files)

In [12]:
len(bam_files), bam_files[0]

(15,
 '/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall/test/good_bams/AT2_EW_36_sorted.bam')

In [13]:
testfb_dir = os.path.join(test_dir,'fb')

In [38]:
cd $test_dir

/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall/test


**reindex all bamfiles**

In [50]:
for b in bam_files:
    environ['b'] = b
    !samtools index $b

**make a bam list file**

In [53]:
def make_bam_list(bam_files):
    with open("bam_files.txt", "w") as o:
        for b in bam_files:
            #bam = b.split('/')[11] 
            #o.write("good_bams/%s \n" % (bam)) 
            o.write("%s \n" % (b)) 

In [54]:
make_bam_list(bam_files)

**make ploidy_list.txt**  
sample_name copy_number

In [84]:
## loop through *.fai write out contig -1 -1 ID ploidy
fai = pd.read_csv(assembly + '.fai',header=None,sep='\t')

In [93]:
contigs = fai.iloc[:,0].tolist()
len(contigs),contigs[:5]

(61970,
 ['dDocent_Contig_6',
  'dDocent_Contig_15',
  'dDocent_Contig_18',
  'dDocent_Contig_20',
  'dDocent_Contig_23'])

In [96]:
## loop through *.fai write out contig -1 -1 ID ploidy
def make_ploidy_list(bam_files):
    with open("ploidy_list.txt", "w") as o:
        for c in contigs:
            for b in bam_files:
                bam = b.split('/')[11]
                ID = bam.split('_sorted.bam')[0] ### This too
                ploidy = ID.split('_')[0]
                ploidy = re.sub(r'\D','',ploidy)
                #print(b)
                #print(ploidy)
                #o.write("good_bams/%s\t%s\n" % (bam,ploidy)) 
                o.write("%s\t-1\t-1\t%s\t%s\n" % (c,ID,ploidy))

In [97]:
make_ploidy_list(bam_files)

In [98]:
#ploidy list should be contigs*bam
len(contigs)*len(bam_files)

929550

In [99]:
!wc -l ploidy_list.txt

929550 ploidy_list.txt


**run freebayes_sh** 

In [57]:
### select options for slurm submission
#account = 'cpu-s5-denovo-0'
#partition = 'cpu-core-0'
account = 'cpu-s1-bionres-0'
partition = 'cpu-s1-bionres-0'
jobname = 'fb_test'
time = '1-00:00:00' #time limit 1 day
cpus = 1
mem_cpu = 80000
email = 'tfaske@nevada.unr.edu'

In [100]:
def write_fb_sh(account,partition,time,jobname,cpus,mem_cpu,email,assembly):
    with open("run_fb_test.sh", "w") as o:
            o.write("""#!/usr/bin/env bash
#SBATCH --account=%s
#SBATCH --partition=%s
#SBATCH --time=%s
#SBATCH --ntasks 1
#SBATCH --cpus-per-task %d
#SBATCH --mem-per-cpu=%d
#SBATCH --job-name fb_test
#SBATCH --output output_fb_test.txt
#SBATCH --mail-type=FAIL,END
#SBATCH --mail-user=%s \n
    
freebayes -b good_bams/*.bam --cnv-map ploidy_list.txt -f %s -v fb_test.vcf -V\n
""" % (account,partition,time,int(cpus),int(mem_cpu),email,assembly))
            

In [101]:
write_fb_sh(account,partition,time,jobname,cpus,mem_cpu,email,assembly)

### Run run_fb_test_sh locally
    cd /data/gpfs/assoc/denovo/tfaske/sagebrush/SNPcall/test
    source activate py36
    bash run_fb_test.sh

## SNP CALL with freebayes FORREAL

call from dDocent: freebayes -b split.1.bam -t mapped.1.bed -v raw.1.vcf -f reference.fasta -m 5 -q 5 -E 3 --min-repeat-entropy 1 -V --populations popmap -n 10 2> fb.1.error.log

**settings to consider**:
  - -L --bam-list FILE A file containing a list of BAM files to be analyzed.
  - -A --cnv-map FILE
                   Read a copy number map from the BED file FILE, which has
                   the format:
                      reference sequence, start, end, sample name, copy number
        ex: 
        chrX -1 -1 NA12878 2  # means NA12878 has copy number 2 in all of chrX
        chrX -1 -1 NA12879 1  # ... copy number 1
        chrX -1 -1 NA13469 2
        chrX -1 -1 NA21328 2
        chrX -1 -1 NA12970 2
        chrY -1 -1 NA12877 1
        chrX -1 -1 NA12877 1
        chr20 20000 30000 NA12877 1   # NA12877 has one copy of chr20:20000..30000
        chr20 20000 30000 NA12879 3   # NA12879 has four copies of chr20:20000..30000


In [102]:
bwa_dir = os.path.join(snp_dir,'good_bams')

In [103]:
cd $bwa_dir

/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall/good_bams


In [104]:
bam_files = []
files = !find . -type f -name '*sorted.bam'
files = [os.path.abspath(x) for x in files if 'bam' in x]
for x in files:
    bam_files.append(x)
bam_files = sorted(bam_files)

In [105]:
len(bam_files), bam_files[0]

(695,
 '/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall/good_bams/AT2_BB_10_sorted.bam')

In [106]:
cd $snp_dir

/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall


**reindex all bamfiles**

In [107]:
for b in bam_files:
    environ['b'] = b
    !samtools index $b

**make a bam list file**

In [108]:
def make_bam_list(bam_files):
    with open("bam_files.txt", "w") as o:
        for b in bam_files:
            #bam = b.split('/')[11] 
            #o.write("good_bams/%s \n" % (bam)) 
            o.write("%s \n" % (b)) 

In [109]:
make_bam_list(bam_files)

**make ploidy_list.txt**  
sample_name copy_number

In [110]:
## loop through *.fai write out contig -1 -1 ID ploidy
fai = pd.read_csv(assembly + '.fai',header=None,sep='\t')

In [111]:
contigs = fai.iloc[:,0].tolist()
len(contigs),contigs[:5]

(61970,
 ['dDocent_Contig_6',
  'dDocent_Contig_15',
  'dDocent_Contig_18',
  'dDocent_Contig_20',
  'dDocent_Contig_23'])

In [114]:
## loop through *.fai write out contig -1 -1 ID ploidy
def make_ploidy_list(bam_files):
    with open("ploidy_list.txt", "w") as o:
        for c in contigs:
            for b in bam_files:
                bam = b.split('/')[10]
                ID = bam.split('_sorted.bam')[0] ### This too
                ploidy = ID.split('_')[0]
                ploidy = re.sub(r'\D','',ploidy)
                #print(b)
                #print(ploidy)
                #o.write("good_bams/%s\t%s\n" % (bam,ploidy)) 
                o.write("%s\t-1\t-1\t%s\t%s\n" % (c,ID,ploidy))

In [115]:
make_ploidy_list(bam_files)

In [116]:
#ploidy list should be contigs*bam
len(contigs)*len(bam_files)

43069150

In [117]:
!wc -l ploidy_list.txt

43069150 ploidy_list.txt


**run freebayes_sh** 

In [135]:
### select options for slurm submission
account = 'cpu-s5-denovo-0'
partition = 'cpu-core-0'
#account = 'cpu-s1-bionres-0'
#partition = 'cpu-s1-bionres-0'
jobname = 'fb_full'
time = '3-00:00:00' #time limit 10 day
cpus = 1
mem_cpu = 100000
email = 'tfaske@nevada.unr.edu'

In [136]:
def write_fb_sh(account,partition,time,jobname,cpus,mem_cpu,email,assembly):
    with open("run_fb_full.sh", "w") as o:
            o.write("""#!/usr/bin/env bash
#SBATCH --account=%s
#SBATCH --partition=%s
#SBATCH --time=%s
#SBATCH --ntasks 1
#SBATCH --cpus-per-task %d
#SBATCH --mem-per-cpu=%d
#SBATCH --job-name fb_full
#SBATCH --output output_fb_full.txt
#SBATCH --mail-type=FAIL,END
#SBATCH --mail-user=%s \n
    
freebayes -b good_bams/*.bam --cnv-map ploidy_list.txt -f %s -v fb_full.vcf -V -d -i -u \n
""" % (account,partition,time,int(cpus),int(mem_cpu),email,assembly))
            

In [137]:
write_fb_sh(account,partition,time,jobname,cpus,mem_cpu,email,assembly)

### Run run_fb_sh locally
    cd /data/gpfs/assoc/denovo/tfaske/sagebrush/SNPcall
    source activate py36
    bash run_fb_full.sh